### Advanced Solutions & Patterns

These solutions go beyond the basic approaches by adding:
- input validation and **type-safety** (type hints)
- discussion of common **edge cases** and pitfalls
- alternative, more **pythonic** or **scalable** patterns (table-driven code, `bisect`, and `match`-`case`)
- quick, lightweight **tests**

#### Exercise 1
Given a variable `a` (containing any value), re-assign the value `'N/A'` if `a` is `None`, and leave `a` unchanged otherwise.

**Why `is None` matters**
- Avoid `a or 'N/A'`: values like `0`, `0.0`, `''`, `False` are *falsy* but **not** missing.
- Use an explicit `is None` check.

In [1]:
# Basic, explicit if/else (safe for falsy values)
from typing import Any

def coalesce_none_to_na(a: Any) -> Any:
    if a is None:
        return "N/A"
    else:
        return a

# Demo
samples = [None, 0, 0.0, "", False, "Python"]
[coalesce_none_to_na(x) for x in samples]

['N/A', 0, 0.0, '', False, 'Python']

In [2]:
# Ternary (conditional expression) — concise and equally safe
def coalesce_none_to_na_ternary(a: Any) -> Any:
    return "N/A" if a is None else a

# Demo
[coalesce_none_to_na_ternary(x) for x in samples]

['N/A', 0, 0.0, '', False, 'Python']

**Common Pitfall (don’t do this):**
```python
a = a or "N/A"  # WRONG for this task
```
This would convert `0`, `''`, and `False` to `'N/A'`, which is **not** what we want.

#### Exercise 2
Do the same thing as Exercise 1, but this time **use a ternary operator**.

In [3]:
# Already shown above, but packaged for clarity
def coalesce_none_to_na_ternary_strict(a):
    return "N/A" if a is None else a

# Quick checks
assert coalesce_none_to_na_ternary_strict(None) == "N/A"
assert coalesce_none_to_na_ternary_strict(0) == 0
assert coalesce_none_to_na_ternary_strict("") == ""
assert coalesce_none_to_na_ternary_strict(False) is False
"ok"

'ok'

#### Exercise 3
Map a **credit score** to a **rating**:

- [0, 580) → Poor
- [580, 670) → Fair
- [670, 740) → Good
- [740, 800) → Very Good
- [800, 850] → Excellent

**Advanced notes**
- Validate range (0–850). Decide what to do for invalid inputs (raise or return a sentinel like `'Invalid'`).
- Prefer **table-driven** code when ranges or labels may change.
- For efficient range lookup, use **`bisect`** (binary search).

In [4]:
from bisect import bisect_left
from typing import Tuple

# Table-driven with boundaries (lower bounds) and labels
BOUNDS: Tuple[int, ...] = (0, 580, 670, 740, 800)
LABELS: Tuple[str, ...] = ("Poor", "Fair", "Good", "Very Good", "Excellent")

def credit_rating(score: int) -> str:
    if not isinstance(score, int):
        raise TypeError("score must be an int")
    if score < 0 or score > 850:
        return "Invalid"
    idx = bisect_left(BOUNDS, score)
    idx = min(idx, len(BOUNDS) - 1)
    if score < BOUNDS[idx]:
        idx -= 1
    return LABELS[idx]

# Sanity checks
tests = {
    0: "Poor", 579: "Poor",
    580: "Fair", 669: "Fair",
    670: "Good", 739: "Good",
    740: "Very Good", 799: "Very Good",
    800: "Excellent", 850: "Excellent",
}
for s, expected in tests.items():
    assert credit_rating(s) == expected, (s, credit_rating(s), expected)
assert credit_rating(-1) == "Invalid"
assert credit_rating(851) == "Invalid"
"credit_rating: all checks passed ✅"

'credit_rating: all checks passed ✅'

In [5]:
# Alternative: explicit conditions (clear for readers)
def credit_rating_readable(score: int) -> str:
    if score < 0 or score > 850:
        return "Invalid"
    if score >= 800:
        return "Excellent"
    elif score >= 740:
        return "Very Good"
    elif score >= 670:
        return "Good"
    elif score >= 580:
        return "Fair"
    else:
        return "Poor"

# Python 3.10+ pattern matching for ranges (using guards)
def credit_rating_match(score: int) -> str:
    match score:
        case s if s < 0 or s > 850:
            return "Invalid"
        case s if s >= 800:
            return "Excellent"
        case s if s >= 740:
            return "Very Good"
        case s if s >= 670:
            return "Good"
        case s if s >= 580:
            return "Fair"
        case _:
            return "Poor"

# Quick parity check among implementations
for s in (0, 100, 579, 580, 669, 670, 739, 740, 799, 800, 850):
    assert credit_rating(s) == credit_rating_readable(s) == credit_rating_match(s)
"alternate impls consistent ✅"

'alternate impls consistent ✅'

#### Exercise 4
Given an `elapsed` time (in seconds), set `magnitude` according to:
- `< 60` → `'seconds'`
- `>= 60 and < 3600` → `'minutes'`
- `>= 3600 and < 86400` → `'hours'`
- `>= 86400 and < 604800` → `'days'`
- `>= 604800` → `'weeks'`

**Advanced notes**
- Validate non-negative integers.
- Prefer named constants for readability.
- Table-driven/bisect scales easily if you add months/years later.

In [6]:
SEC_PER_MIN = 60
SEC_PER_HOUR = 60 * SEC_PER_MIN
SEC_PER_DAY = 24 * SEC_PER_HOUR
SEC_PER_WEEK = 7 * SEC_PER_DAY

In [7]:
def elapsed_magnitude(elapsed: int) -> str:
    if not isinstance(elapsed, int):
        raise TypeError("elapsed must be an int (seconds)")
    if elapsed < 0:
        raise ValueError("elapsed must be non-negative")

    if elapsed < SEC_PER_MIN:
        return "seconds"
    elif elapsed < SEC_PER_HOUR:
        return "minutes"
    elif elapsed < SEC_PER_DAY:
        return "hours"
    elif elapsed < SEC_PER_WEEK:
        return "days"
    else:
        return "weeks"

# Demos
demo_values = [0, 59, 60, 3599, 3600, 86399, 86400, 604799, 604800, 9999999]
[(v, elapsed_magnitude(v)) for v in demo_values]

[(0, 'seconds'),
 (59, 'seconds'),
 (60, 'minutes'),
 (3599, 'minutes'),
 (3600, 'hours'),
 (86399, 'hours'),
 (86400, 'days'),
 (604799, 'days'),
 (604800, 'weeks'),
 (9999999, 'weeks')]

In [8]:
from bisect import bisect_right

# Table-driven variant (easier to extend)
THRESHOLDS = (SEC_PER_MIN, SEC_PER_HOUR, SEC_PER_DAY, SEC_PER_WEEK)
LABELS_TIME = ("seconds", "minutes", "hours", "days", "weeks")

def elapsed_magnitude_bisect(elapsed: int) -> str:
    if not isinstance(elapsed, int):
        raise TypeError("elapsed must be an int (seconds)")
    if elapsed < 0:
        raise ValueError("elapsed must be non-negative")
    idx = bisect_right(THRESHOLDS, elapsed)
    return LABELS_TIME[idx]

# Confirm both versions match
for v in demo_values:
    assert elapsed_magnitude(v) == elapsed_magnitude_bisect(v)
"elapsed magnitude: both implementations match ✅"

'elapsed magnitude: both implementations match ✅'

---
### Quick Tests

In [9]:
# Exercise 1 & 2: strict coalescing (None only)
assert coalesce_none_to_na(None) == "N/A"
assert coalesce_none_to_na(0) == 0
assert coalesce_none_to_na("") == ""
assert coalesce_none_to_na(False) is False
assert coalesce_none_to_na_ternary(None) == "N/A"

# Exercise 3: boundary tests (already done above, repeat a few)
for s, exp in {0:"Poor", 580:"Fair", 670:"Good", 740:"Very Good", 800:"Excellent", 850:"Excellent"}.items():
    assert credit_rating(s) == exp

# Exercise 4: boundaries
assert elapsed_magnitude(0) == 'seconds'
assert elapsed_magnitude(59) == 'seconds'
assert elapsed_magnitude(60) == 'minutes'
assert elapsed_magnitude(3599) == 'minutes'
assert elapsed_magnitude(3600) == 'hours'
assert elapsed_magnitude(86399) == 'hours'
assert elapsed_magnitude(86400) == 'days'
assert elapsed_magnitude(604799) == 'days'
assert elapsed_magnitude(604800) == 'weeks'

print("All advanced checks passed ✅")

All advanced checks passed ✅
